# Raw SQL queries with sqlalchemy and sqlite databases
*Author: Marco Prenassi*   
*License: Creative Commons Attribution 4.0 International*    
<span style="color:red">Beware: Raw sql injection is NOT a secure practice, this is only for learning purposes!!! </span>

### STEP 1: Module setup
*In this first part consider all the python code as "boilerplate code", or... a magic spell*   
*The important part is focusing on install and test the right modules, we will explain later the various components*

In [62]:
# Let's download sqlalchemy and import it as sql
!pip install sqlalchemy

In [63]:
import sqlalchemy as sql

#### Engine definition
*Create an engine with a sqlite protocol into the folder ./databases with the name "simple_db.db", set echo=True to see every interaction with the database*   
*If not present, create the folder ./databases*

In [64]:
import os
engine = sql.create_engine('sqlite:///databases/simple_db.db', echo=True)
if not os.path.isdir('databases'):
    print("directory /databases not found. Making one...")
    !mkdir databases

*Create the database and connect through connect() and after that close the connection*   


In [65]:
try:
    engine.connect().close()
    # completely dispose of the connection pool
    # engine.dispose()
except sql.exc.OperationalError as e:
    print(e)

### STEP 2: Create relations (tables)
*Let's start to test raw SQL*

In [66]:
# a little bit of cleanup before starting to create tables, for now, this is still a magic spell!
with engine.connect() as connection:
    # If we have been naughty and already created a table named users, let's delete it (but only... IF EXISTS)
    raw_sql = "DROP TABLE IF EXISTS users"
    connection.execute(sql.text(raw_sql))
    connection.commit()
    # if you see: sqlalchemy.engine.Engine COMMIT at the end the database has been changed

2024-10-09 07:20:46,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:46,617 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS users
2024-10-09 07:20:46,618 INFO sqlalchemy.engine.Engine [generated in 0.00414s] ()
2024-10-09 07:20:46,623 INFO sqlalchemy.engine.Engine COMMIT


In [67]:
# Ok, this is the start of our database, first step, we create the schema of our first table
# Let's define a simple, generic (this is really important for the next lessons), sample table
with engine.connect() as connection:
    # sample is the name of the table, usually all lowercaps (for some dbms, e.g., postgresql, this is mandandatory
    raw_sql = "CREATE TABLE IF NOT EXISTS samples "
    # We declare the columns inside a ()
    # sample_id is the name, INTEGER is the type, and then are the constrains, like here we set
    # we also chose sample_id as a PRIMARY KEY, so the system will guarantee the uniqueness of the column (among other things) 
    raw_sql += "(sample_id INTEGER PRIMARY KEY, "
    # the we start to put some data, I have chosen for now: weight, and must be not null
    raw_sql += "weight REAL NOT NULL, "
    # and the researcher_id of the researcher, to identify who put the sample inside the system
    raw_sql += "researcher_id VARCHAR(64))"
    # Then we execute the query, and lastly we commit the changes
    connection.execute(sql.text(raw_sql))
    connection.commit()

2024-10-09 07:20:49,149 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:49,152 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS samples (sample_id INTEGER PRIMARY KEY, weight REAL NOT NULL, researcher_id VARCHAR(64))
2024-10-09 07:20:49,153 INFO sqlalchemy.engine.Engine [generated in 0.00475s] ()
2024-10-09 07:20:49,183 INFO sqlalchemy.engine.Engine COMMIT


In [68]:
 # Now let's take our "SCHEMA" and fill it with data ("INSTANCE")
try:
    with engine.connect() as connection:
        raw_sql = "INSERT INTO samples (sample_id, weight, researcher_id) VALUES (0, 12.2 ,'joesmith')"
        connection.execute(sql.text(raw_sql))
        connection.commit()
except sql.exc.IntegrityError as e:
    print("\n--> ERROR: If you see me probably you are violating the primary key constraint, i.e., you are trying to insert a new line with the same pk \n")
    print(e)
    

2024-10-09 07:20:53,103 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:53,108 INFO sqlalchemy.engine.Engine INSERT INTO samples (sample_id, weight, researcher_id) VALUES (0, 12.2 ,'joesmith')
2024-10-09 07:20:53,110 INFO sqlalchemy.engine.Engine [generated in 0.00739s] ()
2024-10-09 07:20:53,116 INFO sqlalchemy.engine.Engine COMMIT


In [69]:
# Let's update our data
with engine.connect() as connection:
    # We change our researcher_id but...
    raw_sql = "UPDATE samples SET researcher_id = 'janesmith' "
    # We need to tell to our DBMS the sample that we want to change!
    raw_sql += "WHERE sample_id=0"
    result = connection.execute(sql.text(raw_sql))
    connection.commit()

2024-10-09 07:20:55,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:55,615 INFO sqlalchemy.engine.Engine UPDATE samples SET researcher_id = 'janesmith' WHERE sample_id=0
2024-10-09 07:20:55,618 INFO sqlalchemy.engine.Engine [generated in 0.00719s] ()
2024-10-09 07:20:55,626 INFO sqlalchemy.engine.Engine COMMIT


In [70]:
# Let's bulk insert some other data
with engine.connect() as connection:
    raw_sql = "INSERT INTO samples ('sample_id','weight','researcher_id') "
    raw_sql += "VALUES (1,9.2,'johnatansmith'),(2,11.3,'johnatansmith'), (3,2.2,'johnatansmith'), (4,14.5,'janesmith')"
    result = connection.execute(sql.text(raw_sql))
    connection.commit()

2024-10-09 07:20:57,706 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:57,724 INFO sqlalchemy.engine.Engine INSERT INTO samples ('sample_id','weight','researcher_id') VALUES (1,9.2,'johnatansmith'),(2,11.3,'johnatansmith'), (3,2.2,'johnatansmith'), (4,14.5,'janesmith')
2024-10-09 07:20:57,728 INFO sqlalchemy.engine.Engine [generated in 0.02267s] ()
2024-10-09 07:20:57,736 INFO sqlalchemy.engine.Engine COMMIT


### STEP 3: QUERYING OPERATIONS
*Now that we have created our first instance of a table, let's check on it and retrieve our data*

In [71]:
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT * FROM samples"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id: {row.sample_id} - weight: {row.weight} - researcher_id: {row.researcher_id}")

2024-10-09 07:20:59,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:20:59,967 INFO sqlalchemy.engine.Engine SELECT * FROM samples
2024-10-09 07:20:59,970 INFO sqlalchemy.engine.Engine [generated in 0.01077s] ()
sample_id: 0 - weight: 12.2 - researcher_id: janesmith
sample_id: 1 - weight: 9.2 - researcher_id: johnatansmith
sample_id: 2 - weight: 11.3 - researcher_id: johnatansmith
sample_id: 3 - weight: 2.2 - researcher_id: johnatansmith
sample_id: 4 - weight: 14.5 - researcher_id: janesmith
2024-10-09 07:20:59,989 INFO sqlalchemy.engine.Engine ROLLBACK


In [72]:
# Select columns
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT samples.sample_id, samples.weight FROM samples"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id: {row.sample_id} - weight: {row.weight}")

2024-10-09 07:21:02,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:02,018 INFO sqlalchemy.engine.Engine SELECT samples.sample_id, samples.weight FROM samples
2024-10-09 07:21:02,019 INFO sqlalchemy.engine.Engine [generated in 0.00231s] ()
sample_id: 0 - weight: 12.2
sample_id: 1 - weight: 9.2
sample_id: 2 - weight: 11.3
sample_id: 3 - weight: 2.2
sample_id: 4 - weight: 14.5
2024-10-09 07:21:02,021 INFO sqlalchemy.engine.Engine ROLLBACK


In [73]:
# Give some sorting conditions with WHERE
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT samples.sample_id, samples.weight FROM samples "
    raw_sql += "WHERE samples.weight > 10"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id: {row.sample_id} - weight: {row.weight}")

2024-10-09 07:21:03,972 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:03,974 INFO sqlalchemy.engine.Engine SELECT samples.sample_id, samples.weight FROM samples WHERE samples.weight > 10
2024-10-09 07:21:03,975 INFO sqlalchemy.engine.Engine [generated in 0.00319s] ()
sample_id: 0 - weight: 12.2
sample_id: 2 - weight: 11.3
sample_id: 4 - weight: 14.5
2024-10-09 07:21:03,980 INFO sqlalchemy.engine.Engine ROLLBACK


In [74]:
# ORDER BY
# Let's order our data, for example, ascending weights
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT samples.sample_id, samples.weight FROM samples "
    raw_sql += "ORDER BY samples.weight ASC"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id: {row.sample_id} - weight: {row.weight}")

2024-10-09 07:21:06,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:06,914 INFO sqlalchemy.engine.Engine SELECT samples.sample_id, samples.weight FROM samples ORDER BY samples.weight ASC
2024-10-09 07:21:06,915 INFO sqlalchemy.engine.Engine [generated in 0.00433s] ()
sample_id: 3 - weight: 2.2
sample_id: 1 - weight: 9.2
sample_id: 2 - weight: 11.3
sample_id: 0 - weight: 12.2
sample_id: 4 - weight: 14.5
2024-10-09 07:21:06,918 INFO sqlalchemy.engine.Engine ROLLBACK


In [75]:
# Let's count our data
# How many sample has the researcher inserted?
# COUNT and GROUP BY
# Let's count our data, grouping it by the resarcher
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT COUNT(samples.sample_id), samples.researcher_id FROM samples "
    raw_sql += "GROUP BY samples.researcher_id"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id counts: {row[0]} - researcher_id: {row.researcher_id}")


2024-10-09 07:21:10,206 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:10,209 INFO sqlalchemy.engine.Engine SELECT COUNT(samples.sample_id), samples.researcher_id FROM samples GROUP BY samples.researcher_id
2024-10-09 07:21:10,209 INFO sqlalchemy.engine.Engine [generated in 0.00372s] ()
sample_id counts: 2 - researcher_id: janesmith
sample_id counts: 3 - researcher_id: johnatansmith
2024-10-09 07:21:10,214 INFO sqlalchemy.engine.Engine ROLLBACK


### Let's ALTER our table schema.
*and learn about the pitfalls on doing so!*

In [77]:
# This works fine
with engine.connect() as connection:
    raw_sql = "ALTER TABLE samples "
    raw_sql += "RENAME COLUMN 'weight' to 'molecular_weight';"
    result = connection.execute(sql.text(raw_sql))
    connection.commit()

2024-10-09 07:21:41,497 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:41,501 INFO sqlalchemy.engine.Engine ALTER TABLE samples RENAME COLUMN 'weight' to 'molecular_weight';
2024-10-09 07:21:41,502 INFO sqlalchemy.engine.Engine [generated in 0.00512s] ()
2024-10-09 07:21:41,603 INFO sqlalchemy.engine.Engine COMMIT


In [78]:
# This have a little problem!
with engine.connect() as connection:
    raw_sql = "ALTER TABLE samples "
    raw_sql += "ADD COLUMN 'magic_data';"
    result = connection.execute(sql.text(raw_sql))
    connection.commit()

# QUIZ: What is it?

2024-10-09 07:21:43,482 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:43,486 INFO sqlalchemy.engine.Engine ALTER TABLE samples ADD COLUMN 'magic_data';
2024-10-09 07:21:43,488 INFO sqlalchemy.engine.Engine [generated in 0.00702s] ()
2024-10-09 07:21:43,500 INFO sqlalchemy.engine.Engine COMMIT


In [79]:
with engine.connect() as connection:
    raw_sql = "ALTER TABLE samples "
    raw_sql += "ADD COLUMN 'magic_data_2' INTEGER DEFAULT 0;"
    result = connection.execute(sql.text(raw_sql))
    connection.commit()


2024-10-09 07:21:45,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:45,718 INFO sqlalchemy.engine.Engine ALTER TABLE samples ADD COLUMN 'magic_data_2' INTEGER DEFAULT 0;
2024-10-09 07:21:45,719 INFO sqlalchemy.engine.Engine [generated in 0.00390s] ()
2024-10-09 07:21:45,726 INFO sqlalchemy.engine.Engine COMMIT


In [80]:
# Let's check!
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT * FROM samples"
    result = connection.execute(sql.text(raw_sql))
    for row in result:
        print(f"sample_id: {row.sample_id} - weight: {row.molecular_weight} - researcher_id: {row.researcher_id} - magic_data_2: {row.magic_data_2}")
        

2024-10-09 07:21:49,049 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:49,051 INFO sqlalchemy.engine.Engine SELECT * FROM samples
2024-10-09 07:21:49,052 INFO sqlalchemy.engine.Engine [cached since 49.09s ago] ()
sample_id: 0 - weight: 12.2 - researcher_id: janesmith - magic_data_2: 0
sample_id: 1 - weight: 9.2 - researcher_id: johnatansmith - magic_data_2: 0
sample_id: 2 - weight: 11.3 - researcher_id: johnatansmith - magic_data_2: 0
sample_id: 3 - weight: 2.2 - researcher_id: johnatansmith - magic_data_2: 0
sample_id: 4 - weight: 14.5 - researcher_id: janesmith - magic_data_2: 0
2024-10-09 07:21:49,054 INFO sqlalchemy.engine.Engine ROLLBACK


In [81]:
# Ok magical_data is fun but we do not need it, let's drop those columns
with engine.connect() as connection:
    # Two operations...
    raw_sql = "ALTER TABLE samples "
    raw_sql += "DROP COLUMN 'magic_data'"
    result = connection.execute(sql.text(raw_sql))
    raw_sql = "ALTER TABLE samples "
    raw_sql += "DROP COLUMN 'magic_data_2'"
    result = connection.execute(sql.text(raw_sql))
    # In a single transaction!
    connection.commit()

2024-10-09 07:21:55,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:21:55,291 INFO sqlalchemy.engine.Engine ALTER TABLE samples DROP COLUMN 'magic_data'
2024-10-09 07:21:55,293 INFO sqlalchemy.engine.Engine [generated in 0.00352s] ()
2024-10-09 07:21:55,316 INFO sqlalchemy.engine.Engine ALTER TABLE samples DROP COLUMN 'magic_data_2'
2024-10-09 07:21:55,317 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()
2024-10-09 07:21:55,322 INFO sqlalchemy.engine.Engine COMMIT


In [82]:
# Let's check!
with engine.connect() as connection:
    # a simple selection of data, * means all the columns of the table
    raw_sql = "SELECT * FROM samples"
    result = connection.execute(sql.text(raw_sql))
    try:
        for row in result:
            print(f"sample_id: {row.sample_id} - weight: {row.molecular_weight} - researcher_id: {row.researcher_id} - magic_data_2: {row.magic_data_2}")
    except AttributeError as e:
        print(f"{e} .... as expected!")

2024-10-09 07:22:01,501 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:22:01,507 INFO sqlalchemy.engine.Engine SELECT * FROM samples
2024-10-09 07:22:01,511 INFO sqlalchemy.engine.Engine [cached since 61.55s ago] ()
Could not locate column in row for column 'magic_data_2' .... as expected!
2024-10-09 07:22:01,524 INFO sqlalchemy.engine.Engine ROLLBACK


*Good work, let's return to the slides!*

### STEP 4: 
*Let's recreate the example on the slide with the different joins*

In [83]:
# Let's create and populate the table1 and table2 of the example, with columns A,B,C and D,E,A
with engine.connect() as connection:
    connection.execute(sql.text(f"CREATE TABLE IF NOT EXISTS table1 (A INTEGER, B INTEGER, C INTEGER)"))
    # DELETE deletes only the data, not the entire table, with "WHERE" could be used to delete only some rows
    connection.execute(sql.text(f"DELETE FROM table1"))
    connection.execute(sql.text(f"INSERT INTO table1 ('A') VALUES (1),(2)"))
    connection.execute(sql.text(f"CREATE TABLE IF NOT EXISTS table2 (D INTEGER, E INTEGER, A INTEGER)"))
    connection.execute(sql.text(f"DELETE FROM table2"))
    connection.execute(sql.text(f"INSERT INTO table2 ('A') VALUES (2),(3)"))

    connection.commit()

2024-10-09 07:22:07,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-09 07:22:07,891 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS table1 (A INTEGER, B INTEGER, C INTEGER)
2024-10-09 07:22:07,893 INFO sqlalchemy.engine.Engine [generated in 0.00381s] ()
2024-10-09 07:22:07,900 INFO sqlalchemy.engine.Engine DELETE FROM table1
2024-10-09 07:22:07,900 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ()
2024-10-09 07:22:07,903 INFO sqlalchemy.engine.Engine INSERT INTO table1 ('A') VALUES (1),(2)
2024-10-09 07:22:07,903 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ()
2024-10-09 07:22:07,904 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS table2 (D INTEGER, E INTEGER, A INTEGER)
2024-10-09 07:22:07,905 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
2024-10-09 07:22:07,917 INFO sqlalchemy.engine.Engine DELETE FROM table2
2024-10-09 07:22:07,921 INFO sqlalchemy.engine.Engine [generated in 0.00426s] ()
2024-10-09 07:22:07,931 INFO sqlalchemy.

In [85]:
engine.echo = False;
with engine.connect() as connection:
    # Let's set up our tables
    # table1:
    print("table1 schema:")
    # PRAGMA tells some information about the database itself, in this case, the schema of our tables
    print("Table1 and Table2:")
    for row in connection.execute(sql.text(f"PRAGMA table_info(table1)")):
        print(row)
    print("table1 instance:")
    print("A,B,C")
    for row in connection.execute(sql.text(f"SELECT * FROM table1")):
        print(row)
    # table2:
    print("\ntable2 schema:")
    for row in connection.execute(sql.text(f"PRAGMA table_info(table1)")):
        print(row)
    print("table1 instance:")
    print("D,E,A")
    for row in connection.execute(sql.text(f"SELECT * FROM table2")):
        print(row)

    # Natural JOIN
    print("\nNATURAL JOIN instance:")
    print("A,B,C,D,E")
    for row in connection.execute(sql.text(f"SELECT * FROM table1 NATURAL JOIN table2")):
        print(row)

    # Inner JOIN
    print("\nINNER JOIN instance:")
    print("table1.A,B,C,D,E,table2.A")
    for row in connection.execute(sql.text(f"SELECT * FROM table1 INNER JOIN table2 ON table1.A = table2.A")):
        print(row)

    # Left JOIN
    print("\nLEFT JOIN instance:")
    print("table1.A,B,C,D,E,table2.A")
    for row in connection.execute(sql.text(f"SELECT * FROM table1 LEFT JOIN table2 ON table1.A = table2.A")):
        print(row)

    # Right JOIN
    print("\n RIGHT JOIN instance:")
    print("table1.A,B,C,D,E,table2.A")
    for row in connection.execute(sql.text(f"SELECT * FROM table1 RIGHT JOIN table2 ON table1.A = table2.A")):
        print(row)

    # Full Outer JOIN
    print("\n FULL OUTER JOIN instance:")
    print("table1.A,B,C,D,E,table2.A")
    for row in connection.execute(sql.text(f"SELECT * FROM table1 FULL OUTER JOIN table2 ON table1.A = table2.A")):
        print(row)



table1 schema:
Table1 and Table2:
(0, 'A', 'INTEGER', 0, None, 0)
(1, 'B', 'INTEGER', 0, None, 0)
(2, 'C', 'INTEGER', 0, None, 0)
table1 instance:
A,B,C
(1, None, None)
(2, None, None)

table2 schema:
(0, 'A', 'INTEGER', 0, None, 0)
(1, 'B', 'INTEGER', 0, None, 0)
(2, 'C', 'INTEGER', 0, None, 0)
table1 instance:
D,E,A
(None, None, 2)
(None, None, 3)

NATURAL JOIN instance:
A,B,C,D,E
(2, None, None, None, None)

INNER JOIN instance:
table1.A,B,C,D,E,table2.A
(2, None, None, None, None, 2)

LEFT JOIN instance:
table1.A,B,C,D,E,table2.A
(1, None, None, None, None, None)
(2, None, None, None, None, 2)

 RIGHT JOIN instance:
table1.A,B,C,D,E,table2.A
(2, None, None, None, None, 2)
(None, None, None, None, None, 3)

 FULL OUTER JOIN instance:
table1.A,B,C,D,E,table2.A
(1, None, None, None, None, None)
(2, None, None, None, None, 2)
(None, None, None, None, None, 3)


## FINAL EXERCISES
* DROP table1 and table2
* CREATE a Sample Table with parameters that you value in your experiments
* CREATE an Instrumentation Table with useful parameters regarding your experiment
* ALTER your Sample Table adding a column regarding the primary key of your Instrumentation Table (add only one per row now)
* INSERT Data in your tables (at least three rows each!)
* UPDATE one row of one table
* SELECT one useful parameter for each table and SELECT + JOIN of those two parameters
* COUNT samples GROUPing them BY their instrument
* Play
  